# Create a delta edge configuration package 

In this notebook, the main goal is to create a delta package from 2 edge configuration packages.  
This is practical if the edge configuration package is large and you want to save time during deploying a small update.  
Let us assume you have already created and deployed an edge configuration package and only want to replace the entrypoint Python script.

To go through this scenario in this notebook, we will need:
- **State-Identifier-edge_1.zip** created in notebook [30-CreatePipelinePackage.ipynb](30-CreatePipelinePackage.ipynb)


### Imports  

In [ ]:
from simaticai import deployment
from pathlib import Path
import shutil
import sys
import os
sys.path.insert(0, str(Path('../src').resolve()))

### Create an updated edge package with a different Python script

This step creates a new edge package with same name as the origin package but with a different entrypoint and different pipeline version.  
**Please note that we only updated the version of the pipeline and left the version of the component untouched.**  
Otherwise, AI Inference Server would regard the whole component changed so we would not benefit from the delta mechanism.

In [ ]:
component = deployment.PythonComponent(name='inference', version='1.0.0', python_version='3.11')

shutil.copy("../entrypoint.py", "../starter.py")
component.add_resources("..", "starter.py")
component.set_entrypoint('starter.py') 
component.add_resources("..", ["src/si"])
component.add_resources("..", "models/clustering-model.joblib")

component.add_input("ph1", "Double")
component.add_input("ph2", "Double")
component.add_input("ph3", "Double")

component.add_output("prediction", "Integer")

#To assure compatibility with older versions of AI SDK (<v1.5.0), you must define the `version` parameter in the `from_components()` method
pipeline = deployment.Pipeline.from_components([component], name='State Identifier')
pipeline.add_parameter("step_size", 300, "Integer")
pipeline.set_timeshifting_periodicity(250)

pipeline.add_dependencies(['tsfresh'])

pipeline.validate()

pipeline_package_path = pipeline.export('../packages')

### Create the delta for the updated package

In [ ]:
origin_package_path = Path('../packages/State-Identifier-edge_1.zip')
new_package_path = Path('../packages/State-Identifier-edge_2.zip')

delta_package_path = deployment.create_delta_package(origin_package_path, new_package_path)

### Results

Compare by how much the delta package is smaller than the complete updated package.  
This correlates with how much faster the upload to AI Inference Server will be.

In [ ]:
origin_package = os.stat('../packages/State-Identifier-edge_1.zip')
delta_package = os.stat('../packages/State-Identifier-edge_delta_2.zip')
print(f'Origin edge package size in Bytes: {origin_package.st_size}\n')
print(f'Delta edge package size in Bytes: {delta_package.st_size}\n')